In [9]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider("https://polygon-mainnet.g.alchemy.com/v2/cEHzRdgL5rGydq_YIokK2"))
assert w3.is_connected(), "RPC failed"

# USDC token on Polygon
token_address = Web3.to_checksum_address("0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174")
aave_pool = Web3.to_checksum_address("0x7F23D86Ee20D869112572136221e173428DD740B")

ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function"
    }
]

usdc = w3.eth.contract(address=token_address, abi=ERC20_ABI)
balance = usdc.functions.balanceOf(aave_pool).call()
decimals = usdc.functions.decimals().call()

normalized = balance / (10 ** decimals)

print(f"USDC Balance in Aave V3 Polygon: {normalized:,.2f}")


ModuleNotFoundError: No module named 'web3'

In [3]:
import pandas as pd

df = pd.read_csv("lending_protocols_by_chain.csv")

# Filter for Polygon-only deployments
polygon_units = df[df["chains"].str.lower() == "polygon"]

print("Polygon protocol-chain units:")
print(polygon_units[["name", "id"]])

Polygon protocol-chain units:
             name    id
11        Aave V3  1599
42      Morpho V1  4025
56    Compound V3  2088
65  Fluid Lending  4167
97        Aave V2   111


In [5]:
from chain_registry import ChainRegistry

reg = ChainRegistry()                 # fetches Chainlist      
print(reg.get_rpc(1))                 # → one RPC for Ethereum mainnet (chainId=1)


https://0xrpc.io/eth
https://api.zan.top/base-mainnet


In [6]:
import pandas as pd

csv_path = "lending_protocols_by_chain.csv"  # adjust if needed

df = pd.read_csv(csv_path)
protocols = sorted(df["name"].dropna().unique())

print(f"{len(protocols)} distinct protocols:")
for protocol in protocols:
    print(protocol)


41 distinct protocols:
Aave V2
Aave V3
AlphaLend
Benqi Lending
Capyfi
Compound V2
Compound V3
Curve LlamaLend
Dolomite
Echelon Market
Echo Lending
EtherFi Borrowing Market
Euler V2
Fluid Lending
Gearbox
Hydration Lending
HyperLend Pooled
HypurrFi Pooled
Jupiter Lend
JustLend
Kamino Lend
Kinetic
Lista Lending
Maple
Moonwell Lending
Morpho V0 AaveV3
Morpho V1
NAVI Lending
Rhea Lend
Save
Silo V2
SparkLend
Suilend
Sumer.money
Takara Lend
Tectonic
Tydro
Venus Core Pool
YeiLend
cap
marginfi Lending


In [12]:
from web3 import Web3

rpc = "https://polygon-mainnet.g.alchemy.com/v2/cEHzRdgL5rGydq_YIokK2"
w3 = Web3(Web3.HTTPProvider(rpc))
addr = Web3.to_checksum_address("0xd05e3E715d945B59290df0ae8eF85c1BdB684744")
code = w3.eth.get_code(addr)
print(len(code))

ModuleNotFoundError: No module named 'web3'

In [ ]:
import pandas as pd
from test_aave_polygon_export import get_polygon_prices  # your working price fetcher

def main():
    # Load reserves CSV (from your working export)
    csv_path = "aave_v3_polygon_reserves_2025-10-28.csv"
    df = pd.read_csv(csv_path)

    # Get all unique underlying contract addresses
    addresses = df["underlying_address"].dropna().unique().tolist()

    # Fetch live prices from your existing function
    print(f"📊 Fetching prices for {len(addresses)} assets from CoinGecko...")
    prices = get_polygon_prices(addresses)  # should return {address_lower: price_usd}

    # Map prices to dataframe
    df["price_usd"] = df["underlying_address"].str.lower().map(prices).fillna(0.0)

    # Compute per-token TVL = aToken_supply × price_usd
    df["tvl_usd"] = df["aToken_supply"] * df["price_usd"]

    # Summate TVL
    total_tvl = df["tvl_usd"].sum()

    print("\n🔹 Aave V3 Polygon TVL Breakdown:")
    print(df[["underlying_symbol", "aToken_supply", "price_usd", "tvl_usd"]].sort_values("tvl_usd", ascending=False).to_string(index=False))
    print(f"\n💰 Total TVL (USD): ${total_tvl:,.2f}")

    # Save enriched CSV
    out_path = csv_path.replace(".csv", "_tvl.csv")
    df.to_csv(out_path, index=False)
    print(f"\n✅ TVL written to: {out_path}")

if __name__ == "__main__":
    main()
